In [0]:
SET TIME ZONE 'Pacific/Auckland';

#GAS

In [0]:
with L1 (
  select ParticipantCode, Date, max(SubmissionDate) as SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresstoragegas group by all
)
select a.ParticipantCode, a.Date,a.SubmissionDate,
a.StoredVolumePetajoules, a.MaxWithdrawalRateTerajoulesPerDay, a.MaxInjectionRateTerajoulesPerDay, a.MaxStorageCapacityTerajoules
from datahub.wholesale.thermalfueldisclosuresstoragegas a
inner join L1 b
on (a.ParticipantCode = b.ParticipantCode 
and a.Date = b.Date 
and a.SubmissionDate = b.SubmissionDate)
order by ParticipantCode, Date


In [0]:

with L1 (
  select ParticipantCode, MonthEnded, max(SubmissionDate) as SubmissionDate
  from datahub.wholesale.thermalfueldisclosurescontractsgas group by all
)
select a.ParticipantCode,a.MonthEnded,a.SubmissionDate,ContractedGasMinPetajoulesPerMonth,ContractedGasMaxPetajoulesPerMonth,DeliverabilityMinTerajoulesPerDay,DeliverabilityMaxTerajoulesPerDay
from  datahub.wholesale.thermalfueldisclosurescontractsgas a
inner join L1 b
on (a.ParticipantCode = b.ParticipantCode 
and a.MonthEnded = b.MonthEnded 
and a.SubmissionDate = b.SubmissionDate)
--where a.SubmissionDate >= '2025-09-05'
order by ParticipantCode, MonthEnded

In [0]:
with L1 (
  select ParticipantCode, Max(Date) as latestDate
  from datahub.wholesale.thermalfueldisclosuresstoragegas group by all
), L2 as (
  select a.ParticipantCode, a.Date, Max(a.SubmissionDate) as SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresstoragegas a inner join L1 on (a.Date = L1.latestDate and a.ParticipantCode = L1.ParticipantCode)
  group by all
)
select b.*, a.StoredVolumePetajoules, a.MaxWithdrawalRateTerajoulesPerDay, a.MaxInjectionRateTerajoulesPerDay, a.MaxStorageCapacityTerajoules
from datahub.wholesale.thermalfueldisclosuresstoragegas a
inner join L2 b
on (a.ParticipantCode = b.ParticipantCode 
and a.Date = b.Date 
and a.SubmissionDate = b.SubmissionDate)
order by a.ParticipantCode, a.Date

In [0]:
with CTCT as (
  SELECT 'CTCT' as ParticipantCode, `Date`, '2025-04-02' as SubmissionDate, 
  StoredVolume_PJ,MaxWithdrawlRate_TJperDay,MaxInjectionRate_TJperDay, 3500 as MaxStorageCapacity_TJ
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/GasStorageContact.csv`
  WITH ( header = 'true', inferSchema = 'true' )
), TODD as (
  SELECT 'TODD' as ParticipantCode, `Date`, '2025-04-02' as SubmissionDate, 
  StoredVolume_PJ,MaxWithdrawlRate_TJperDay,MaxInjectionRate_TJperDay,3500 as MaxStorageCapacity_TJ
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/GasStorageTODD.csv`
  WITH ( header = 'true', inferSchema = 'true' )
), DataHub as (
  select a.ParticipantCode, a.Date,a.SubmissionDate,
  StoredVolumePetajoules, a.MaxWithdrawalRateTerajoulesPerDay, a.MaxInjectionRateTerajoulesPerDay, a.MaxStorageCapacityTerajoules
  from datahub.wholesale.thermalfueldisclosuresstoragegas a
), L1 as (
  select * from DataHub union select * from CTCT union select * from TODD
), L2 (
  select ParticipantCode, Date, max(SubmissionDate) as SubmissionDate
  from L1 group by all
)
-- select * from L1
select a.ParticipantCode, a.Date,a.SubmissionDate,
a.StoredVolumePetajoules, a.MaxWithdrawalRateTerajoulesPerDay, 
coalesce(a.MaxInjectionRateTerajoulesPerDay,a.MaxWithdrawalRateTerajoulesPerDay) as MaxInjectionRateTerajoulesPerDay , 
a.MaxStorageCapacityTerajoules
from L1 a
inner join L2 b
on (a.ParticipantCode = b.ParticipantCode 
and a.Date = b.Date 
and a.SubmissionDate = b.SubmissionDate)
where a.ParticipantCode <> 'GENE'
order by `Date`, ParticipantCode, SubmissionDate

In [0]:
with CTCT as (
  SELECT 'CTCT' as ParticipantCode, *
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/ContractedGasCTCT.csv`
  WITH ( header = 'true', inferSchema = 'true' )
), GENE as (
  SELECT 'GENE' as ParticipantCode, *
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/ContractedGasGENE.csv`
  WITH ( header = 'true', inferSchema = 'true' )
), TODD as (
  SELECT 'TODD' as ParticipantCode, *
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/ContractedGasTODD.csv`
  WITH ( header = 'true', inferSchema = 'true' )
), L1 as (select * from CTCT union select * from GENE union select * from TODD),
L2 as (
select ParticipantCode, last_day(`Month`) as MonthEnded, SubmissionDate,
ContractedGasMin_PJperMonth as ContractedGasMinPetajoulesPerMonth,
ContractedGasMax_PJperMonth as ContractedGasMaxPetajoulesPerMonth,
DeliverabilityMin_TJperday as DeliverabilityMinTerajoulesPerDay,
DeliverabilityMax_TJperday as DeliverabilityMaxTerajoulesPerDay
from L1
union
select ParticipantCode,MonthEnded,SubmissionDate,
ContractedGasMinPetajoulesPerMonth,ContractedGasMaxPetajoulesPerMonth,
DeliverabilityMinTerajoulesPerDay,DeliverabilityMaxTerajoulesPerDay
from  datahub.wholesale.thermalfueldisclosurescontractsgas
), L3 (
  select ParticipantCode, MonthEnded, max(SubmissionDate) as SubmissionDate
  from L2 group by all
)
, GasContracts as (
select a.ParticipantCode,a.MonthEnded,a.SubmissionDate,
ContractedGasMinPetajoulesPerMonth,ContractedGasMaxPetajoulesPerMonth,
Cast(coalesce(DeliverabilityMinTerajoulesPerDay,0) as double) as DeliverabilityMinTerajoulesPerDay,
Cast(coalesce(DeliverabilityMaxTerajoulesPerDay,0) as double) as DeliverabilityMaxTerajoulesPerDay
from L2 a
inner join L3 b
on (a.ParticipantCode = b.ParticipantCode 
and a.MonthEnded = b.MonthEnded 
and a.SubmissionDate = b.SubmissionDate)
)
select ParticipantCode, cast(date_trunc('month', MonthEnded) as Date) as MonthStarted, MonthEnded, SubmissionDate,
ContractedGasMinPetajoulesPerMonth as MinPJPerMonth,
ContractedGasMaxPetajoulesPerMonth as MaxPJPerMonth,
DeliverabilityMinTerajoulesPerDay as MinTJPerDay,
DeliverabilityMaxTerajoulesPerDay as MaxTJPerDay,
round(greatest(ContractedGasMinPetajoulesPerMonth * 1000/Day(MonthEnded), DeliverabilityMinTerajoulesPerDay),3) as DailyMinTJ,
round(least(ContractedGasMaxPetajoulesPerMonth * 1000/Day(MonthEnded), NULLIF(DeliverabilityMaxTerajoulesPerDay, 0)),3) as DailyMaxTJ

from GasContracts
order by MonthEnded, ParticipantCode

In [0]:
select DateSupplied as TradingDate, ParticipantCode, 
max(PriceExclCarbonDollarsPerGigajoule) as max_price,
min(PriceExclCarbonDollarsPerGigajoule) as min_price,
sum(PriceExclCarbonDollarsPerGigajoule*VolumeGigajoules)/sum(VolumeGigajoules) as VWAP
from ea_prd.gold.factthermalfueldisclosurestransactionsgas
where DateSupplied >= '2025-01-01' and VolumeGigajoules > 0
and Counterparty in ('Sport Market','Spot Market','Spot market','Tradepoint')
and TransactionType in ('Purchase','Purchase '
                      --  ,'Sale','Sell'
                       ) 
group by all
Order by ParticipantCode, TradingDate

Databricks visualization. Run in Databricks to view.

In [0]:
select TradingDate, GasVWAPDollarsPerGigajoule from datahub.wholesale.gasandcarbonpriceindices 
where TradingDate >= '2020-01-01'
order by TradingDate

Databricks visualization. Run in Databricks to view.

#COAL

In [0]:
with Coal as (
  SELECT 'GENE' as ParticipantCode, `Date`, `Date`+Interval 1 day as SubmissionDate, FuelName as Fuel,
  20.54 as `EnrgFactor_GJ/tonne`, TotalVolumeStored_kt as Storage_kTon, 
  round(DeliveriesReceivedInLastWeek_kt,3) as LastWeekDelivery_kTon, 
  round(DeliveryPriceExclCarbon_dollarsperkilotonne/1000,2) as `CO2exclusive_$/tonne`,
  VolumeOptions_kt as BurningOptions_kTon
 --,*
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/CoalStorageAndReceived.csv`
  WITH ( header = 'true', inferSchema = 'true' )
)
select * from Coal


In [0]:
SELECT 'GENE' as ParticipantCode, `Date`, `Date`+Interval 1 day as SubmissionDate, FuelName as Fuel,
  20.54 as `EnrgFactor_GJ/tonne`, TotalVolumeStored_kt as Storage_kTon, 
  round(DeliveriesReceivedInLastWeek_kt,3) as LastWeekDelivery_kTon, 
  round(DeliveryPriceExclCarbon_dollarsperkilotonne/1000,2) as `CO2exclusive_$/tonne`,
  VolumeOptions_kt as BurningOptions_kTon
 ,*
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/CoalStorageAndReceived.csv`
  WITH ( header = 'true', inferSchema = 'true' )

In [0]:
with Coal as (
  SELECT 'GENE' as ParticipantCode, `Date`, `Date`+Interval 1 day as SubmissionDate, FuelName as Fuel,
  20.54 as `EnrgFactor_GJ/tonne`, TotalVolumeStored_kt as Storage_kTon, 
  round(DeliveriesReceivedInLastWeek_kt,3) as LastWeekDelivery_kTon, 
  round(DeliveryPriceExclCarbon_dollarsperkilotonne/1000,2) as `CO2exclusive_$/tonne`,
  VolumeOptions_kt as BurningOptions_kTon
 --,*
  FROM csv.`/Volumes/datahub/marketmonitoring/marketmonitoring/Sam/s46data/DataSummary/CoalStorageAndReceived.csv`
  WITH ( header = 'true', inferSchema = 'true' )
), L1 as (
select * from Coal
union
select ParticipantCode, Date, SubmissionDate, Fuel,
case 
  when SpecificEnergyMegaJoulesPerTonne < 100 then SpecificEnergyMegaJoulesPerTonne 
  else SpecificEnergyMegaJoulesPerTonne/1000
end as `EnrgFactor_GJ/tonne`,
TotalVolumeStoredKilotonnes as Storage_kTon,
DeliveriesReceivedInLastWeekKilotonnes as LastWeekDelivery_kTon,
case 
  when DeliveryPriceExclCarbonDollarsPerKilotonne > 10000 then round(DeliveryPriceExclCarbonDollarsPerKilotonne/10000,2)
  else DeliveryPriceExclCarbonDollarsPerKilotonne
end as `CO2exclusive_$/tonne`,
VolumeOptionsKilotonnes as BurningOptions_kTon
from datahub.wholesale.thermalfueldisclosuresstoragesolid
)
select ParticipantCode,Date,SubmissionDate,Fuel,
round(1000 * Storage_kTon * `EnrgFactor_GJ/tonne`,3) as Storeage_GJ,
round(`CO2exclusive_$/tonne`/`EnrgFactor_GJ/tonne`,2) as CO2exclusiveDollarsPer_GJ,
`EnrgFactor_GJ/tonne`,
Storage_kTon,
BurningOptions_kTon,
LastWeekDelivery_kTon,
`CO2exclusive_$/tonne`
from L1
order by Date


In [0]:
with L1 as (select ParticipantCode, Date, SubmissionDate, Fuel,
case 
  when SpecificEnergyMegaJoulesPerTonne < 100 then SpecificEnergyMegaJoulesPerTonne 
  else SpecificEnergyMegaJoulesPerTonne/1000
end as `EnrgFactor_GJ/tonne`,
TotalVolumeStoredKilotonnes as Storage_kTon,
DeliveriesReceivedInLastWeekKilotonnes as LastWeekDelivery_kTon,
case 
  when DeliveryPriceExclCarbonDollarsPerKilotonne > 10000 then round(DeliveryPriceExclCarbonDollarsPerKilotonne/10000,2)
  else DeliveryPriceExclCarbonDollarsPerKilotonne
end as `CO2exclusive_$/tonne`,
VolumeOptionsKilotonnes as BurningOptions_kTon
from datahub.wholesale.thermalfueldisclosuresstoragesolid )

select ParticipantCode,Date,SubmissionDate,Fuel,
round(1000 * Storage_kTon * `EnrgFactor_GJ/tonne`,3) as Storeage_GJ,
round(`CO2exclusive_$/tonne`/`EnrgFactor_GJ/tonne`,2) as CO2exclusiveDollarsPer_GJ,
`EnrgFactor_GJ/tonne`,
Storage_kTon,
BurningOptions_kTon,
LastWeekDelivery_kTon,
`CO2exclusive_$/tonne`
from L1
--where Date > '2025-04-09'
order by Date

In [0]:
with L1 (
  select ParticipantCode, Date, max(SubmissionDate) as SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresdeliveriessolid 
  -- where Date >= current_date()
  group by all
)
select a.ParticipantCode, a.Date, a.SubmissionDate,a.Fuel, a.DeliveryVolumeKilotonnes*20.540 as DeliveryVolumeGJ, 
a.DeliveryVolumeKilotonnes as DeliveryVolumnkT
from datahub.wholesale.thermalfueldisclosuresdeliveriessolid a
inner join L1 b
on (a.ParticipantCode = b.ParticipantCode 
and a.Date = b.Date 
and a.SubmissionDate = b.SubmissionDate)
order by ParticipantCode, Date

#DIESEL

In [0]:
with L1 (
  select ParticipantCode, Date, max(SubmissionDate) as SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresstoragediesel group by all
)
select a.ParticipantCode, a.Date,a.SubmissionDate,a.StorageLocation,
round(a.StoredDieselLitres * a.SpecificEnergyGigajoulesPerLitre,3) as Storage_GJ,
round(a.DeliveriesReceivedInLastWeekLitres * a.SpecificEnergyGigajoulesPerLitre,3) as LastWeekDelivery_GJ,
round(1000*a.DeliveryPriceExclCarbonDollarsPerLitre * a.SpecificEnergyGigajoulesPerLitre, 2) as CO2ExclusiveDollarsPerGJ,
a.StoredDieselLitres, a.SpecificEnergyGigajoulesPerLitre,
a.DeliveriesReceivedInLastWeekLitres, a.DeliveryPriceExclCarbonDollarsPerLitre

from datahub.wholesale.thermalfueldisclosuresstoragediesel a
inner join L1 b
on (a.ParticipantCode = b.ParticipantCode 
and a.Date = b.Date 
and a.SubmissionDate = b.SubmissionDate)
order by ParticipantCode, Date

In [0]:
with L1 (
  select ParticipantCode, Max(Date) as latestDate
  from datahub.wholesale.thermalfueldisclosuresstoragediesel group by all
), L2 as (
  select a.ParticipantCode, a.Date, Max(a.SubmissionDate) as SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresstoragediesel a inner join L1 on (a.Date = L1.latestDate and a.ParticipantCode = L1.ParticipantCode)
  group by all
)
select a.ParticipantCode, a.Date,a.SubmissionDate,a.StorageLocation,
round(a.StoredDieselLitres * a.SpecificEnergyGigajoulesPerLitre,3) as Storage_GJ,
round(a.DeliveriesReceivedInLastWeekLitres * a.SpecificEnergyGigajoulesPerLitre,3) as LastWeekDelivery_GJ,
round(a.DeliveryPriceExclCarbonDollarsPerLitre/ a.SpecificEnergyGigajoulesPerLitre,2) as `LastWeekDelivery_$/GJ`,
a.StoredDieselLitres, a.SpecificEnergyGigajoulesPerLitre,
a.DeliveriesReceivedInLastWeekLitres, a.DeliveryPriceExclCarbonDollarsPerLitre
from datahub.wholesale.thermalfueldisclosuresstoragediesel a
inner join L2 b
on (a.ParticipantCode = b.ParticipantCode 
and a.Date = b.Date 
and a.SubmissionDate = b.SubmissionDate)
order by a.ParticipantCode, a.Date

In [0]:
with L1 (
  select ParticipantCode, MonthEnded, max(SubmissionDate) as SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresdeliveriesdiesel
  where SubmissionDate >= '2025-04-23'
  group by all
)
select a.ParticipantCode, a.MonthEnded, a.SubmissionDate,a.DeliveryVolumeLitres as DeliveryVolumnLitres
from datahub.wholesale.thermalfueldisclosuresdeliveriesdiesel a
inner join L1 b
on (a.ParticipantCode = b.ParticipantCode 
and a.MonthEnded = b.MonthEnded 
and a.SubmissionDate = b.SubmissionDate)
order by ParticipantCode, MonthEnded

#WIP

In [0]:
select * from datahub.wholesale.spdnodal where TradingDate = '2025-01-01' and CaseTypeCode = 'RTD' and Unit

In [0]:
with L1 as (
  select TradingDate, TradingPeriodNumber, MegawattsByGenerationType/2 as MWh,
  floor((datediff(TradingDate, date_trunc('year', TradingDate)) / 7) + 1) as WeekOfYear
  from datahub.wholesale.emissionsbygenerationtype
  where TradingDate >= '2024-01-01' and GenerationType = 'Coal'
), Coal as (
  select year(TradingDate) as `Year`, case when WeekOfYear > 52 then 52 else WeekOfYear end as `Week`, round(sum(MWh),3) as MWh
  from L1 group by 1,2
), L2 as (
  select TradingDate, TradingPeriodNumber, sum(KilowattHours/1000) as MWh,
  floor((datediff(TradingDate, date_trunc('year', TradingDate)) / 7) + 1) as WeekOfYear
  from datahub.wholesale.reconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and PointOfConnectionCode = 'HLY2201'
  and TraderCode = 'GENE' and FlowDirection = 'Injection'
  group by 1,2
), Total as (
select year(TradingDate) as `Year`, case when WeekOfYear > 52 then 52 else WeekOfYear end as `Week`, round(sum(MWh),3) as MWh
from L2 group by 1,2
)
select Total.Year, Total.Week, Total.MWh as TotalMWh, Coal.MWh as CoalMWh, Total.MWh - Coal.MWh as GasMWh 
from Total inner join Coal using (`Year`,`Week`)
order by 1, 2

In [0]:
with L1 as (
  select TradingDate, TradingPeriodNumber, sum(KilowattHours/1000) as MWh,
  floor((datediff(TradingDate, date_trunc('year', TradingDate)) / 7) + 1) as WeekOfYear
  from datahub.wholesale.reconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and PointOfConnectionCode = 'SFD2201'
  and TraderCode = 'CTCT' and FlowDirection = 'Injection'
  group by 1,2
) 
select year(TradingDate) as `Year`, case when WeekOfYear > 52 then 52 else WeekOfYear end as `Week`, round(sum(MWh),3) as MWh
from L1 group by 1,2
order by 1,2


In [0]:
with L1 as (
  select TradingDate, TradingPeriodNumber, sum(KilowattHours/1000) as MWh,
  floor((datediff(TradingDate, date_trunc('year', TradingDate)) / 7) + 1) as WeekOfYear
  from datahub.wholesale.reconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and PointOfConnectionCode in ('JRD1101','MKE1101')
  and TraderCode = 'TODD' and FlowDirection = 'Injection'
  group by 1,2
) 
select year(TradingDate) as `Year`, case when WeekOfYear > 52 then 52 else WeekOfYear end as `Week`, round(sum(MWh),3) as MWh
from L1 group by 1,2
order by 1,2

In [0]:
with L1 as (
  select
  TradingDate,TradingPeriodNumber,IntervalDateTime, 
  PointofConnectionCode, sum(SPDInitialMegawatts) as InitialMW, sum(SPDMegawatts) as GenerationMW
  from datahub.wholesale.spdnodal 
  where TradingDate >= '2024-01-01' and CaseTypeCode = 'RTD' and UnitCode <> "N/A"
  and Fuel in ('Coal','Gas','Coal/Gas') 
  -- and PointOfConnectionCode in ('JRD1101','MKE1101')
  and PointOfConnectionCode in ('HLY2201')
  and Technology <> 'Cogeneration'
  group by all
), L2 as (
  select TradingDate,TradingPeriodNumber, PointofConnectionCode, avg(InitialMW) as InitialMW, avg(GenerationMW) as GenerationMW,
  floor((datediff(TradingDate, date_trunc('year', TradingDate)) / 7) + 1) as WeekOfYear
  from L1 group by all
  order by all
)
select year(TradingDate) as `Year`, case when WeekOfYear > 52 then 52 else WeekOfYear end as `Week`, 
round(avg(GenerationMW)/2,3) as MW, round(sum(GenerationMW)/2,3) as MWh
from L2 group by 1,2
order by 1,2

In [0]:
select Date as TradingDate, Fuel, DeliveryVolumeKilotonnes as Delivery_kTonnes,SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresdeliveriessolid

In [0]:
with L1 as (
  select Date as TradingDate, max(SubmissionDate) as SubmissionDate
  from ea_prd.gold.factthermalfueldisclosuresdeliveriessolid
  group by all
)
select Date as TradingDate, Fuel, DeliveryVolumeKilotonnes as Delivery_kTonnes,a.SubmissionDate
from ea_prd.gold.factthermalfueldisclosuresdeliveriessolid a 
inner join L1 b on (a.Date = b.TradingDate and a.SubmissionDate = b.SubmissionDate)
order by D

In [0]:
With CoalGeneration as (
  select TradingDate, sum(MegawattsByGenerationType/2) as CoalGeneration_MWh
  from datahub.wholesale.emissionsbygenerationtype
  where TradingDate >= '2025-04-09' and GenerationType = 'Coal'
  group by all
), CoalDelivery as (
  select Date as TradingDate, Fuel, DeliveryVolumeKilotonnes as Delivery_kTonnes,SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresdeliveriessolid
  order by Date
), CoalStorage as (
  select Date  as TradingDate, Fuel, TotalVolumeStoredKilotonnes as Stored_kTonnes,SpecificEnergyMegaJoulesPerTonne as GJperTonne
  from datahub.wholesale.thermalfueldisclosuresstoragesolid
)
  select a.TradingDate, a.CoalGeneration_MWh, b.delivery_ktonnes as CoalDelivery_kTonnes,Stored_kTonnes, GJperTonne
  from CoalGeneration a 
  left join CoalDelivery b on a.TradingDate = b.TradingDate
  left join CoalStorage c on a.TradingDate = c.TradingDate
  order by TradingDate
  

In [0]:
select Date as TradingDate, Fuel, DeliveryVolumeKilotonnes as Delivery_kTonnes,SubmissionDate
  from datahub.wholesale.thermalfueldisclosuresdeliveriessolid
  order by Date

In [0]:
select Date, Fuel, DeliveryVolumeKilotonnes as Delivery_kTonnes,SubmissionDate
from datahub.wholesale.thermalfueldisclosuresdeliveriessolid
order by Date

In [0]:
select * from datahub.wholesale.thermalfueldisclosurestransactionsgas
order by ParticipantCode

In [0]:
with L1 as (
  select TradingDate, TradingPeriodNumber, KilowattHours/500 as MW  from ea_prd.gold.factreconciledinjectionandofftake
where FlowDirection = 'Injection'
and PointOfConnectionCode = 'SFD2201' and TraderCode = 'CTCT'
), L2 as (
  select TradingDate, TradingPeriodNumber, DollarsPerMegawattHour from ea_prd.gold.factpredispatchandfinalenergyprices
  where PointOfConnectionCode = 'SFD2201' and PriceTypeCode = 'F'
)
select L1.* ,L2.dollarspermegawatthour from L1 inner join L2 on (L1.TradingDate = L2.TradingDate and L1.TradingPeriodNumber = L2.TradingPeriodNumber)
where L1.Tradingdate >= '2020-01-01'
order by 1,2

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
with L1 as (
  select TradingDate, TradingPeriodNumber, KilowattHours/500 as MW  from ea_prd.gold.factreconciledinjectionandofftake
where PointOfConnectionCode = 'HLY2201' and FlowDirection = 'Injection'
and TraderCode = 'GENE'
), L2 as (
  select TradingDate, TradingPeriodNumber, DollarsPerMegawattHour from ea_prd.gold.factpredispatchandfinalenergyprices
  where PointOfConnectionCode = 'HLY2201' and PriceTypeCode = 'F'
), L3 as (
select L1.* ,L2.dollarspermegawatthour, ceil(L1.MW/100,0)*100 as Group_MW from L1 inner join L2 on (L1.TradingDate = L2.TradingDate and L1.TradingPeriodNumber = L2.TradingPeriodNumber)
where L1.Tradingdate >= '2020-01-01'
), L4 as (
  select Group_MW, sum(MW*dollarspermegawatthour)/sum(MW) as Group_price from L3
  group by all
)
select Group_MW, ceil(Group_price/10)*10 as Group_price
from L4
where Group_MW > 0
order by 1

In [0]:
with L1 as (
  select TradingDate, TradingPeriodNumber, KilowattHours/500 as MW  from ea_prd.gold.factreconciledinjectionandofftake
where PointOfConnectionCode = 'SFD2201' and FlowDirection = 'Injection'
and TraderCode = 'CTCT'
), L2 as (
  select TradingDate, TradingPeriodNumber, DollarsPerMegawattHour from ea_prd.gold.factpredispatchandfinalenergyprices
  where PointOfConnectionCode = 'SFD2201' and PriceTypeCode = 'F'
), L3 as (
select L1.* ,L2.dollarspermegawatthour, ceil(L1.MW/100,0)*100 as Group_MW from L1 inner join L2 on (L1.TradingDate = L2.TradingDate and L1.TradingPeriodNumber = L2.TradingPeriodNumber)
where L1.Tradingdate >= '2020-01-01'
), L4 as (
  select Group_MW, sum(MW*dollarspermegawatthour)/sum(MW) as Group_price from L3
  group by all
)
select Group_MW, ceil(Group_price/10)*10 as Group_price
from L4
where Group_MW > 0
order by 1

In [0]:
select TradingDate, avg(DollarsPerMegawattHour) as Price from ea_prd.gold.factpredispatchandfinalenergyprices 
where TradingDate between '2024-01-01' and '2024-12-31' and PriceTypeCode = 'F' and PointOfConnectionCode = 'HAY2201' group by TradingDate
order by TradingDate

Databricks visualization. Run in Databricks to view.

In [0]:
select * from ea_prd.gold.dimgenerationplant